Universidade Federal de Alagoas

Instituto de Computação


Disciplina: Sistemas Distribuídos

Semestre letivo: 2018.2

Professor: André Lage Freitas


# Trabalho Prático AB2

**Importante**. Leia atentamente as especificações pois só serão aceitos trabalhos que as obedecerem.

## Pontuação Extra

Atividades feitas:

- Comentários em inglês por todo o código;

- Documentação em inglês em alguns pontos do código.


## Parte 1: Implementação de um monitor de carga da trabalho descentralizado

**Implementar uma versão modificada do detector de falha não confiável** (_unreliable failure detector_) descrito na Seção _15.1.1 Failure assumptions and failure detectors_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

A **modificação** consiste em enviar informações sobre a carga do sistema (_system workload_) em vez de informações sobre falhas que indicariam os processos suspeitos e não-suspeitos. Para isso, em vez de cada processo enviar mensagem do tipo `I'm p and I'm alive`, os processos enviarão a medição interna da sua carga de trabalho. 

Para simular a medição da carga de trabalho dos processos, utilize números aleatórios entre `0` e `1` com precisão de uma casa decimal apenas, por exemplo, `0.4`.

Utilize como período de envio das mensagens `2` segundos.

### Detalhes de implementação

Cada processo do sistema deverá ser implementado como um Julia Worker. O Master será apenas o gestor do experimento, ou seja, será responsável para criar os processos (Julia Workers) e iniciar o sistema. Uma vez iniciado, o sistema deve funcionar por si sem depender do Master; uma vez que o algoritmo é descentralizado.

Para fins de visualização do monitoramento da carga de trabalho, os **Workers devem enviar ao Master** periodicamente (a cada `2` segundos) uma mensagem com o estado dos workers que eles monitoram (incluindo ele mesmo). Essa mensagem deve ser impresso na tela do master da seguinte forma.

```julia
[...]
Workload from Worker 4: [0.4,0.9,0.7]
Workload from Worker 2: [0.4,0.9,0.7]
Workload from Worker 3: [0.4,0.9,0.7]
[...]
```

A última linha acima significa que as cargas de trabalho dos Workers monitorados pelo Worker 3 são `0.4,0.9,0.7`, ou seja, os Workers têm as seguintes carga de trabalho:

* Worker 2 = 0.4
* Worker 3 = 0.9
* Worker 4 = 0.7

Lembre-se de que, em Julia, o ID do Master é 1 e os Workers têm ID maiores que 1.

## Parte 2: Implementação de um algoritmo de Eleição

**Implementar o algoritmo de Eleição** baseado em anel (_ring-based election algorithm_) descrito na Seção _15.3 Elections_ do livro a seguir:

Coulouris, Dollimore, Kindberg and Blair, _Distributed Systems: Concepts and Design_. Addison-Wesley, 2012.

Considere as seguintes condições.

1. Uma eleição nova é disparada **quando a carga de trabalho do líder for maior ou igual a `0.8`** e **o Worker que identificou essa alta carga de trabalho do líder tenha carga de trabalho menor que `0.8`**.
1. Em caso de empate, o Worker com ID mais alto ganha a eleição.
1. Caso não haja Worker com carga de trabalho menor que `0.8` para liderar, escolhe-se o Worker com menor carga de trabalho.

Quando um líder for eleito, ele deve enviar a a seguinte mensagem ao Master:

```julia
[...]
New leader elected! Worker 2
[...]
```


### Suporte de programação distribuída em Julia

Para utilizar o suporte de programação distribuída em Julia, primeiro devemos carergar a biblioteca `Distributed`:

```julia
using Pkg
Pkg.add("Distributed")
using Distributed
```

A seguir, segue uma lista (não extensiva) de funções que poderão ajudar na programação distribuída para a implementação do trabalho:

```julia
addprocs
remotecall
@fetchfrom
@everywhere
sleep
@spawn
```

Para buscar uma rápida ajuda sobre as funções, utilize `?` na frente da função (ver exemplo abaixo). 

```julia
?addprocs
```

Utilize o [material do mini-curso do Professor](https://github.com/proflage/2018-julia-hands-on) para estudo e a [documentação oficial da linguagem Julia (1.x)](https://docs.julialang.org/en/v1/) para demais dúvidas.

## Código-fonte

Documente seu código em comentários no próprio código-fonte.

**Todo o seu código tem que estar colado na próxima célula**, mesmo se ele tenha sido feito em IDEs, células separadas, etc. O Professor testará a apenas o que estiver colado na próxima célula.

In [4]:
# First, importing the Distributed package, which will provide all the functions related to Distributed Systems.
using Distributed
# In case there are previous workers already added from past executions, let's remove them.
rmprocs(workers())
# Adding 3 workers
addprocs(3)
# Below we have all the variables needed in each worker for the workload monitor and leader election to work. From top to bottom:
#
# - `workersState` is a dictionary that contains the known workload of the current worker and others. It's a dictionary
# because it requires less effort when dealing with the recreation of workers. That's because when recreating workers, the old ids
# aren`t used anymore, and they keep growing. Using a dictionary ends up saving memory.
#
# - `electionParticipant` indicates if the current worker is participating in an election
#
# - `leaderId` represents the current known leader for each worker.
#
# - `nextWorkerId` is the id of the next worker in the ring. This value is calculated and stored when the worker starts.
@everywhere workers() global workersState = Dict()
@everywhere workers() global electionParticipant = false
@everywhere workers() global leaderId = -1
@everywhere workers() global nextWorkerId = 0

# The `receiveUpdate` function receives messages from other workers with updates to their workloads.
# The values are stored in the `workersState` dictionary.
@everywhere function receiveUpdate(wid, load)
    global workersState
    workersState[wid] = load
end

# This is just a helper function to convert the dictionary into an array.
@everywhere function getWorkersArray(workersState)
    arr = zeros(0)
    for w in workers()
        val = 0.0
        if haskey(workersState, w)
            val =  workersState[w]
        end
        push!(arr, val)
    end
    return arr
end

# This function is called repeatedly, with an interval of 2 seconds, thanks to the timer we created below.
@everywhere function notifyAndCheckWorkloads(timer)
    # We have to use `global` in order to use the variable we created above. If we removed this, electionParticipant
    # would be created as a local variable.
    global electionParticipant
    # As required by the specification, each worker has a random number rounded to 1 decimal place as its workload.
    workersState[myid()] = round(rand(), digits=1)
    # Broadcast my new workload to all other workers, using the `receiveUpdate` function.
    for w in workers()
        if myid() != w
            remotecall(receiveUpdate, w, myid(), workersState[myid()])
        end
    end
    # Ask Master to display all my known workloads.
    remotecall(updateWorkerState, 1, myid(), getWorkersArray(workersState))
    # These are the conditions that have to be satisfied in order for an election to occur.
    # I can't be a participant of another election, the leader must have a workload greater or equal to 0.8 and my
    # workload must be below 0.8.
    if !electionParticipant && haskey(workersState, leaderId) && workersState[leaderId] >= 0.8 && workersState[myid()] < 0.8
        # If the conditions are satisfied, we start a new election.
        # We become a participant and tell my neighbor about the election.
        electionParticipant = true
        remotecall(electionMessageReceived, nextWorkerId, myid())
    end
end

# This function deals with the initialization of each worker.
@everywhere function startWorker()
    global nextWorkerId
    global leaderId
    # The first leader is the first worker.
    leaderId = sort(workers())[1]
    # We create a timer to call the `notifyAndCheckWorkloads` function repeatedly.
    timer = Timer(notifyAndCheckWorkloads, 0, interval=2)
    wait(timer)
    # Defining the nextWorkerId such that we end up with a ring.
    nextWorkerId = myid() + 1
    if nextWorkerId > sort(workers())[length(workers())]
        nextWorkerId = sort(workers())[1]
    end
end

# Calling this function notifies Master of a new workload dictionary from one of the workers.
@everywhere function updateWorkerState(wid, loadarray)
    println("Workload from Worker $wid: ", loadarray)
end

# This function sends the `elected` message to all workers. It stops when the message reaches the elected worker.
@everywhere function elected(elected_id)
    global electionParticipant
    global leaderId
    # Updating variables to reflect the result of the election.
    leaderId = elected_id
    electionParticipant = false
    # If I'm not the elected worker, send this message to my neighbor.
    if myid() != elected_id
        remotecall(elected, nextWorkerId, elected_id)
    end
end

# This function deals with the election process. It has a single argument, the id of the worker that wants to be the new leader.
@everywhere function electionMessageReceived(candidate_id)
    global electionParticipant
    global leaderId
    # If I receive a message with my id on it, I'm the best choice.
    if candidate_id == myid()
        # So I'll set myself as the new leader.
        leaderId = candidate_id
        electionParticipant = false
        # And broadcast this information to all other workers in the ring.
        remotecall(println, 1, "New leader elected! Worker $candidate_id")
        remotecall(elected, nextWorkerId, candidate_id)
        return
    end
    myWorkload = 0.0
    if haskey(workersState, myid())
        myWorkload = workersState[myid()]
    end
    candidateWorkload = 0.0
    if haskey(workersState, candidate_id)
        candidateWorkload = workersState[candidate_id]
    end
    # In case the candidate has a workload smaller than mine or, if the workloads are the same but my id is smaller,
    # I'll just send the message to the next worker in the ring, keeping the same candidate.
    if candidateWorkload < myWorkload || (myWorkload == candidateWorkload && myid() < candidate_id)
        remotecall(electionMessageReceived, nextWorkerId, candidate_id)
        electionParticipant = true
        return
    end
    # If, however, I have a smaller workload than the current candidate or, if the workloads are equal but my id is higher,
    # I'll be the new candidate. After that, we tell the next worker in the ring about the election, sending our id as 
    # candidate.
    if  !electionParticipant && (myWorkload < candidateWorkload || (myWorkload == candidateWorkload && myid() > candidate_id))
        candidate_id = myid()
        remotecall(electionMessageReceived, nextWorkerId, candidate_id)
        electionParticipant = true
        return
    end
end

# Calling the `startWorker` function for each worker. 

@sync for w in workers()
     @spawnat w startWorker()
end

Workload from Worker 13: [0.6, 0.0, 0.0]
Workload from Worker 11: [0.9, 0.0, 0.6]
Workload from Worker 12: [0.1, 0.9, 0.6]
Workload from Worker 11: [0.7, 0.1, 0.6]
Workload from Worker 13: [0.9, 0.7, 0.1]
New leader elected! Worker 12
Workload from Worker 12: [0.8, 0.7, 0.9]
Workload from Worker 11: [0.5, 0.8, 0.9]
Workload from Worker 13: [0.1, 0.5, 0.8]
New leader elected! Worker 11
Workload from Worker 12: [0.6, 0.5, 0.1]
Workload from Worker 11: [0.8, 0.6, 0.1]
Workload from Worker 13: [0.3, 0.8, 0.6]
New leader elected! Worker 13
Workload from Worker 12: [0.2, 0.8, 0.3]
Workload from Worker 11: [0.4, 0.2, 0.3]
Workload from Worker 13: [0.8, 0.4, 0.2]
Workload from Worker 12: [0.5, 0.4, 0.8]
New leader elected! Worker 11
Workload from Worker 11: [0.7, 0.5, 0.8]
Workload from Worker 13: [0.7, 0.7, 0.5]
Workload from Worker 12: [0.4, 0.7, 0.7]
Workload from Worker 11: [0.7, 0.4, 0.7]
Workload from Worker 13: [0.3, 0.7, 0.4]
Workload from Worker 12: [1.0, 0.7, 0.3]
Workload from Worke


## Entrega

As respostas deverão ser entregues na parte indicada desse arquivo reservada ao código fonte, no formato [IJulia Notebook](https://github.com/JuliaLang/IJulia.jl), que utiliza tecnologia [Jupyter](https://www.jupyter.org).

O programa deve ser implementado na linguagem de programação [**Julia** versão 1.x](https://julialang.org/downloads/). Seu trabalho será testado na [JuliaBox](https://juliabox.com). 

Baixe seu arquivo `.ipynb` e anexe-o ao Google Classroom. Não se esqueça de **testar seu arquivo `.ipynb` na JuliaBox** antes de enviá-lo.


### Forma 

O arquivo IJulia Notebook deverá ser entregue ao Professor **exclusivamente através do Google Classroom**.



A responsabilidade sobre a integridade do arquivo contendo trabalho é exclusivamente dos discentes. Serão ignorados os trabalhos cujos arquivos não conseguirem ser abertos pelo Professor.

### Prazos

O prazo de entrega está descrito no **Google Classroom**.


## Pontuação extra

O(a) discente que realizar mais tarefas, além do que foi especificado neste trabalho, o professor atribuirá de 0,5 a 1,0 ponto extra a depender da relevância da contribuição no programa. O critério será decidido pelo professor. 

O(a) discente deve indicar qual é a tarefa executada. Por exemplo, adição de funcionalidades, armazenamento de dados em arquivo, documentação de código, comentários em inglês sem erros ortográficos, etc.


**Plágio** A nota zero será atribuída caso haja qualquer tipo de cópia parcial ou integral assim como as devidas medidas legais. Leia a [cartilha sobre plágio](http://www.noticias.uff.br/arquivos/cartilha-sobre-plagio-academico.pdf).

## Disclaimer

Esse material foi elaborado pelo [Prof. André Lage Freitas](https://sites.google.com/a/ic.ufal.br/andrelage/) e está licenciado sob a licença _GNU General Public License v3.0_.